In [1]:
import numpy as np
import pandas as pd

In [5]:
inters = pd.read_csv('clean_data/interactions_ml.csv')
votes = pd.read_csv('clean_data/votes_ml.csv')
comments = pd.read_csv('clean_data/comments_ml.csv')
lastpart = pd.read_csv('clean_data/lastpart_ml.csv')
users_index = pd.read_csv('clean_data/target.csv')

# 1. Employee features

The are the features that realte to absolute employee metrics. 

## 2.1 vote features

In [6]:

vote_feats = votes.groupby(['uid'])['vote'].agg({'votes_1': lambda x: len(x[x==1]),
                                                        'votes_2': lambda x: len(x[x==2]),
                                                        'votes_3': lambda x: len(x[x==3]),
                                                        'votes_4': lambda x: len(x[x==4]),
                                                        'votes_mean': lambda x: x.mean(),
                                                        'votes_std': lambda x: x.std(),
                                                       'votes_num': lambda x: len(x),
                                                       }).copy().fillna(0)

## 2.2 Comments features

In [7]:
likes_feats = comments.groupby(['uid'])['likes'].agg({
                                          'likes_mean': lambda x: len(x),
                                          'likes_std': lambda x: x.std(),
                                          'likes_sum': lambda x: x.sum(),
                                                       })
dislikes_feats = comments.groupby(['uid'])['dislikes'].agg({
                                          'dislikes_mean': lambda x: len(x),
                                          'dislikes_std': lambda x: x.std(),
                                          'dislikes_sum': lambda x: x.sum(),
                                                       })
coms_feats = comments.dropna().groupby(['uid'])['comment'].agg({'com_num': lambda x: len(x),
                                                             'com_mean': lambda x: x.apply(len).mean(),
                                                             'com_std': lambda x: x.map(len).std(),
                                                             'com_sum': lambda x: x.map(len).sum()})

## 2.3 Merge all the features

In [8]:
features = pd.merge(vote_feats,likes_feats,left_index=True,right_index=True,how='outer')
features = pd.merge(features,dislikes_feats,left_index=True,right_index=True,how='outer')
features = pd.merge(features,coms_feats,left_index=True,right_index=True,how='outer')
features = pd.merge(features,users_index.set_index('uid')[['churn','comp_short']],left_index=True,right_index=True,how='left').fillna(0)#fill users with 0 comments
features.rename(columns={'comp_short':'comid'},inplace=True)
E_features = features.copy()
E_features.rename(columns=dict([(x,'E_'+str(x)) for x in E_features.columns]),inplace=True)
